This document has been machine translated from Japanese to English.

# Structure search with CrySPY & PFP

How to run CrySPY, a crystal structure search tool, using the PFP force field on Matlantis.

## Pre-setup

Prepare the following libraries.

**CrySPY**
 - https://github.com/Tomoki-YAMASHITA/CrySPY

The latest version at the time of writing is v1.1.1: https://github.com/Tomoki-YAMASHITA/CrySPY/releases/tag/v1.1.1

Installation notes
- Please note that CrySPY version 1.0.0 and later require Python 3.8 or higher.
- If Matlantis Features is already installed, an Error due to version incompatibility of required NumPy and SciPy in both libraries may occur. In this case, reinstall the version required by CrySPY and then the CrySPY installation will complete.

**LAMMPS: using matlantis-lammps**

 - Copy from the experimental package and install according to the ipynb.

In [1]:
# Download CrySPY v1.1.1
!pip install csp-cryspy

 - pyxtal
   - docs: https://pyxtal.readthedocs.io/en/latest/
   - github: https://github.com/qzhu2017/PyXtal

A library for operations considering the symmetry of space groups, which CrySPY uses internally.

You can install it with the following command.

In [2]:
!pip install pyxtal

## Executing the official LAMMPS Example

Running CrySPY involves executing a python program, which is recommended to be done via the Terminal.

By running code like the following on the Terminal, you can execute a LAMMPS integration example that is included with the CrySPY package beforehand.

```
# Activate python=3.8
source ~/.py38/bin/activate

# Move to the directory of the LAMMPS Example (note that the directory location differs for each user)
cd CrySPY/example/LAMMPS_bash_Si16_lj_RS

# Run CrySPY
cryspy
```

※ For the second and subsequent runs, if you wish to initialize before execution, delete any existing files among the data, work directories, and cryspy.out, cryspy.stat, lock_cryspy files before running.

As shown above, by moving to the directory containing the desired configuration files with the `cd` command, and executing `cryspy` from that location, you can run the tool.

`cryspy` will repeatedly check the following according to the progress of the job:

- 1. Generating structure candidates
- 2.1 Moving on to calculate the next structure candidate
- 2.2 Checking if the current structure calculation has finished
- 2. Determining if all calculations have finished

Thus, **you need to regularly call `cryspy`.**

For example, in the case of the above LAMMPS example, if it's functioning correctly, you will see logs like this.

**First `cryspy`:**

```
2023/07/04 09:20:42
CrySPY 1.1.1
Start cryspy.py
Number of MPI processes: 1

Read input file, cryspy.in
Save input data in cryspy.stat

# --------- Generate initial structures
# ------ mindist
Si - Si 1.11
Structure ID      0 was generated. Space group: 203 --> 227 Fd-3m
Structure ID      1 was generated. Space group: 176 --> 176 P6_3/m
Structure ID      2 was generated. Space group:  59 -->  59 Pmmn
Structure ID      3 was generated. Space group:   4 -->   4 P2_1
Structure ID      4 was generated. Space group: 159 --> 159 P31c

Elapsed time for structure generation: 0:00:00.560896
```

→Five candidate structures have been generated. The actual structures generated are random and therefore differ with each execution.


**Second `cryspy`:**

```
2023/07/04 09:21:09
CrySPY 1.1.1
Restart cryspy.py
Number of MPI processes: 1



# ---------- job status
ID      0: submit job, Stage 1
ID      1: submit job, Stage 1
```

Only for structures ID=0 and ID=1, the actual energy calculations (including structure optimization) begin.

**Third `cryspy`:**

```
# ---------- job status
ID      0: Stage 1 Done!
    submitted job, ID      0 Stage 2
ID      1: Stage 1 Done!
    submitted job, ID      1 Stage 2
```

You run cryspy again after some time since the second run (after the structure optimization has finished), and

In this way, you need to run and check `cryspy` multiple times until all calculations are finished.

### Automated execution with the watch command


This operation can be automated using the watch command.

The following command is an example of continuing to run `cryspy` every 20 seconds.

```
watch -n 20 cryspy
```


If you execute this on the Terminal and keep it running, the CrySPY structure search will progress, and ultimately you will receive the following output.

```
2023/07/04 09:28:03
CrySPY 1.1.1
Restart cryspy.py
Number of MPI processes: 1



# ---------- job status
Done all structures!
```

## Executing the Example using PFP


In the above LAMMPS Example, the Si crystal structure search was performed using the LJ potential.<br/>
Let's now try doing the structure search using PFP instead.

In the `LAMMPS_bash_Si16_lj_RS_PFP` directory, you'll find files with settings changed to use PFP.Specifically, `calc_in/in.si_1` and `calc_in/in.si_2` are LAMMPS input configuration files, which have been modified to use the PFP force field like this:


```
pair_style pfp_api v4.0.0
pair_coeff * * species Si
```

Start the Terminal and after moving to the `LAMMPS_bash_Si16_lj_RS_PFP` directory, try running the following command.

```
# Activate python=3.8
source ~/.py38/bin/activate

# Move to the directory of the PFP LAMMPS Example
cd LAMMPS_bash_Si16_lj_RS_PFP

# Run CrySPY (version using the watch command for regular execution)
watch -n 20 cryspy
```

(Note that the installation of `matlantis-lammps` is required for execution.)

※ For the second and subsequent runs, if you wish to initialize before execution, delete any existing files among the data, work directories, and cryspy.out, cryspy.stat, lock_cryspy files before running.

The Example using PFP will take longer for structure optimization and such compared to the LJ potential case, so it will take time for each job to finish, and logs like "ID      0: still queueing or running" will persist for some time (a few minutes to about 10 minutes).
Waiting for a while will complete all the jobs.

### How to verify PFP is running properly

Firstly, **be aware that executing `cryspy` needs to be done via the Terminal.**<br/>

If you execute ``!cryspy` from a cell in the Jupyter Lab Notebook, "ID      0: still queueing or running" will be displayed, but it has been confirmed that PFP gets stuck without running in the background.

On the Terminal, if "ID      1: still queueing or running" continues to be displayed and you're not sure if the execution is proceeding normally, please check the calculation logs in `work/000001/log.struc`, `work/000001/out.si`, `work/000001/stage1_out.si`, etc. to see if there is progress. ("000001" should be changed according to the job number being executed.)